In [1]:
# A scratch cell to run functions directly.
using Reexport
using Distributions
using Statistics
using GeoStats
using Random
using DataFrames
import GLMakie as Mke
using Infiltrator
using AbstractGPs
using Unitful
using Plots
using DataStructures
using POMDPs
include("../src/config.jl");
include("../src/pomdp.jl")
pomdp = CCSPOMDP();


In [3]:
visualize_gt(pomdp, 2)

### Test: Action function returns all actions

In [3]:
all_actions = POMDPs.actions(pomdp);

### Test: Observation function returns correct distributions

In [4]:
POMDPs.observation(pomdp, all_actions[2], pomdp.state)

Vector{MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}}MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}


Distributions.ProductDistribution{2, 1, Vector{MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}}, Distributions.Continuous, Float64}(
dists: MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}[MixtureModel{MvNormal}(K = 3)
components[1] (prior = 0.0000): FullNormal(
dim: 1
μ: [0.4911705015086909]
Σ: [8.99797545553156;;]
)

components[2] (prior = 1.0000): FullNormal(
dim: 1
μ: [0.4911705015086909]
Σ: [8.99797545553156;;]
)

components[3] (prior = 0.0000): FullNormal(
dim: 1
μ: [0.4911705015086909]
Σ: [8.99797545553156;;]
)

, MixtureModel{MvNormal}(K = 3)
components[1] (prior = 0.0000): FullNormal(
dim: 1
μ: [7.7047972537329095]
Σ: [2.2004889975550124;;]
)

components[2] (prior = 1.0000): FullNormal(
dim: 1
μ: [2.306264246397947]
Σ: [2.2004889975550124;;]
)

components[3] (prior = 0.0000): FullNormal(
dim: 1
μ: [0.6143327060556478]
Σ: [2.2004889975550124;;]


In [5]:
POMDPs.reward(pomdp, pomdp.state, all_actions[2])

8.4

In [ ]:
POMDPs.observation(pomdp, all_actions[21], pomdp.state)

Vector{MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}}MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}


Distributions.ProductDistribution{2, 1, Vector{MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}}, Distributions.Continuous, Float64}(
dists: MixtureModel{Multivariate, Distributions.Continuous, MvNormal, Distributions.Categorical{Float64, Vector{Float64}}}[MixtureModel{MvNormal}(K = 3)
components[1] (prior = 0.0000): FullNormal(
dim: 5
μ: [-1.6846216272143124, 1.2588268014093273, -0.04447095044980642, 1.903266270784286, 0.9342941801461393]
Σ: [99.7506234413886 3.383471001640027e-14 … 6.1251230582483484e-40 -8.241087688860867e-53; 3.383471001640027e-14 99.7506234413886 … -4.552448831870191e-27 6.1251230582483484e-40; … ; 6.1251230582483484e-40 -4.552448831870191e-27 … 99.7506234413886 3.383471001640027e-14; -8.241087688860867e-53 6.1251230582483484e-40 … 3.383471001640027e-14 99.7506234413886]
)

components[2] (prior = 1.0000): FullNormal(
dim: 5
μ: [-1.6846216272143124, 1.2588268014093273, -0.04447095044980642, 1.903266

In [9]:
POMDPs.reward(pomdp, pomdp.state, all_actions[21])

83466.34627779722

In [ ]:
visualize_uncertainty(pomdp, 2, :z)

### Test: Points close are predicted with more accuracy and less uncertainty than points far.

In [ ]:
gtlayer = pomdp.state.earth[1].gt

# seismic_line = (x1=0.0, y1=0.0, x2=5.0, y2=98.0)
well1 = Point(19, 34)
well2 = Point(30, 12)
well_close = Point(31, 14)
well_far = Point(98, 97)

all_wells = Multi([well1, well2])
data_at_all_wells = gtlayer[all_wells, :]

γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET)

okrig = GeoStatsModels.OrdinaryKriging(γ)
fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

# We notice that prediction at close has much less variance than prediction at far, 
# and as an additional sanity check the prediction at close is close to the well2 point.
# Krigging predictprob only does one attribute at a time.

probs_far = GeoStatsModels.predictprob(fitkrig, :z, well_far)
probs_close = GeoStatsModels.predictprob(fitkrig, :z, well_close)
probs_close, probs_far

### Test: Uncertainty is low around known points.
Additionally, as nugget is increased uncertainty (both globally and at known points) increases.
Pay attention to colorbar when verifying this.

### Buy well data at 3-4 locations and visualize belief

In [ ]:
buy_well_data(pomdp, 2)
buy_well_data(pomdp, 5)
buy_well_data(pomdp, 7)

In [ ]:
visualize_uncertainty(Multi([pomdp.collected_locs...]), 3, "z") # Note this is time consuming at just 3 gathered points.

In [ ]:
distr = observe(pomdp, Point(18, 24), 3, "z")
rand(distr)

In [7]:
pomdp

CCSPOMDP(CCS_State(LayerFeatures[LayerFeatures(10000×4 GeoTable over 100×100 CartesianGrid, 10000×3 DataFrame
   Row │ z           permeability  topSealThickness 
       │ Float64     Float64       Float64          
───────┼────────────────────────────────────────────
     1 │  0.418264        54.2897           69.1713
     2 │  0.218509        53.86             67.1535
     3 │  0.683682        55.1475           68.8464
     4 │  0.0379527       54.5552           67.4198
     5 │ -1.76864         53.1284           66.0203
     6 │ -0.967533        52.9579           66.1128
     7 │ -1.9599          53.6322           66.4487
     8 │ -1.90968         52.5635           66.814
   ⋮   │     ⋮            ⋮               ⋮
  9994 │  0.283401        52.9514           67.9631
  9995 │  1.2244          53.2844           68.717
  9996 │  0.609048        54.3204           69.3424
  9997 │  1.06244         54.7998           69.6658
  9998 │  2.22551         55.5369           68.2291
  9999 │  2.8